# HW4 Speakers Classification

---

# Get Data

## Download Data (These links are no longer working, I finally downloaded these on Kaggle)

In [50]:
# !curl -L "https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partaa" -o Dataset.tar.gz.partaa

# !curl -L "https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partab" -o Dataset.tar.gz.partab

# !curl -L "https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partac" -o Dataset.tar.gz.partac

# !curl -L "https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partad" -o Dataset.tar.gz.partad

# !copy /b Dataset.tar.gz.part* Dataset.tar.gz

# !tar -zxvf Dataset.tar.gz


## Manually Unzip

---

# Preparation

## Importing

In [9]:
import os
import json
import math
from pathlib import Path

import numpy as np
import pandas as pd
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Optimizer, AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset, random_split

from tqdm.auto import tqdm, trange
import random

from torch.utils.tensorboard import SummaryWriter


## Define Dataset

In [10]:
class MyDataset(Dataset):

    def __init__(self, data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len

        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]

        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]

        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append(
                    [utterances["feature_path"], self.speaker2id[speaker]])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        feat_path, speaker_id = self.data[index]
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        if len(mel) > self.segment_len:
            start = random.randint(0, len(mel) - self.segment_len)
            mel = torch.FloatTensor(mel[start:start + self.segment_len])
        else:
            mel = torch.FloatTensor(mel)

        speaker_id = torch.tensor(speaker_id, dtype=torch.long)
        return mel, speaker_id

    def get_speaker_number(self):
        return self.speaker_num


## Define Dataloader

In [11]:
def collate_batch(batch):
    mel, speaker = zip(*batch)
    mel = pad_sequence(mel, batch_first=True, padding_value=-20)
    speaker = torch.stack(speaker)
    return mel, speaker


def get_dataloader(data_dir_val,
                   batch_size_val,
                   n_workers_val,
                   segment_len_val=128):
    dataset = MyDataset(data_dir_val, segment_len=segment_len_val)
    speaker_num_val = dataset.get_speaker_number()

    train_length = int(0.9 * len(dataset))
    lengths = [train_length, len(dataset) - train_length]
    train_dataset, val_dataset = random_split(dataset, lengths)

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size_val,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers_val,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    valid_loader = DataLoader(
        val_dataset,
        batch_size=batch_size_val,
        shuffle=False,
        drop_last=False,
        num_workers=n_workers_val,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    return train_loader, valid_loader, speaker_num_val


## Define Model

In [12]:
class Classifier(nn.Module):

    def __init__(self, d_model=80, num_speakers=600, dropout=0.1):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
                                                        dim_feedforward=256,
                                                        nhead=2)
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, num_speakers),
        )

    def forward(self, mels):
        """
		args:
			mels: (batch size, length, 40)
		return:
			out: (batch size, num_speakers)
		"""
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        # mean pooling
        stats = out.mean(dim=1)

        # out: (batch, num_speakers)
        out = self.pred_layer(stats)
        return out


## Fixing seed

In [13]:
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


## Hyperparameters

In [14]:
# general paths and settings
data_dir = "./Dataset"
model_save_path = "./speaker_model.ckpt"
submission_path = "./submission.csv"
tensorboard_log_dir = "./runs/speaker_classification"
seed = 3407

# training parameters
batch_size = 32
num_workers = 0
num_speakers = 600  # will be determined from the dataset

# model parameters (transformer related)
d_model = 80
n_head = 2
dim_feedforward = 256
dropout_rate = 0.1
num_encoder_layers = 2

# optimizer and scheduler parameters
learning_rate = 1e-3
num_warmup_steps = 1000
total_steps = 70000

# training control
valid_steps = 2000
early_stop_patience = 10

# apply seed
same_seeds(seed)

# device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


## Training Utilities

In [15]:
def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
):

    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps))
        return max(
            0.0, 0.5 *
            (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))

    return LambdaLR(optimizer, lr_lambda, last_epoch)


def model_fn_speaker(batch, model, criterion_val, device_val):
    mels, labels = batch
    mels = mels.to(device_val)
    labels = labels.to(device_val)

    outs = model(mels)
    loss = criterion_val(outs, labels)

    preds = outs.argmax(1)
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy


def validate_speaker_model(valid_loader_val,
                           model_val,
                           criterion_val,
                           device_val,
                           current_step_or_epoch,
                           writer_tb=None,
                           pbar_desc_prefix=""):
    model_val.eval()
    epoch_valid_loss = 0.0
    epoch_valid_corrects = 0
    num_valid_samples = 0

    batch_pbar = tqdm(valid_loader_val,
                      leave=False,
                      desc=f"{pbar_desc_prefix} Validation")
    with torch.no_grad():
        for features, labels in batch_pbar:
            features = features.to(device_val)
            labels = labels.to(device_val)

            outputs = model_val(features)
            loss = criterion_val(outputs, labels)

            preds = outputs.argmax(dim=-1)
            epoch_valid_loss += loss.item() * features.size(0)
            epoch_valid_corrects += (preds == labels).sum().item()
            num_valid_samples += features.size(0)

            batch_pbar.set_postfix(
                loss=f"{loss.item():.4f}",
                acc=f"{(preds == labels).sum().item()/features.size(0):.2%}")

    avg_epoch_valid_loss = epoch_valid_loss / num_valid_samples if num_valid_samples > 0 else 0
    avg_epoch_valid_acc = epoch_valid_corrects / num_valid_samples if num_valid_samples > 0 else 0

    if writer_tb:
        writer_tb.add_scalar('Loss/valid_step', avg_epoch_valid_loss,
                             current_step_or_epoch)
        writer_tb.add_scalar('Accuracy/valid_step', avg_epoch_valid_acc,
                             current_step_or_epoch)

    model_val.train()
    return avg_epoch_valid_loss, avg_epoch_valid_acc


In [16]:
def train_speaker_model(model_to_train, train_loader_val, valid_loader_val,
                        criterion_val, optimizer_val, scheduler_val,
                        device_val):
    tb_writer = SummaryWriter(log_dir=tensorboard_log_dir)

    best_valid_acc = 0.0
    best_model_state_dict = None
    epochs_no_improve = 0

    train_iterator = iter(train_loader_val)

    epoch_running_loss = 0.0
    epoch_running_corrects = 0
    epoch_num_samples = 0
    completed_data_passes = 0

    pbar_total_steps = tqdm(range(total_steps), desc="Total Training Steps")

    for step in pbar_total_steps:
        model_to_train.train()

        try:
            batch = next(train_iterator)
        except StopIteration:
            if epoch_num_samples > 0:
                current_pass_avg_loss = epoch_running_loss / epoch_num_samples
                current_pass_avg_acc = epoch_running_corrects / epoch_num_samples
                pbar_total_steps.write(
                    f"End of Training Data Pass {completed_data_passes + 1}: "
                    f"Avg Loss: {current_pass_avg_loss:.4f}, Avg Acc: {current_pass_avg_acc:.2%}"
                )
                if tb_writer:
                    tb_writer.add_scalar('Loss/train_data_pass_avg',
                                         current_pass_avg_loss,
                                         completed_data_passes + 1)
                    tb_writer.add_scalar('Accuracy/train_data_pass_avg',
                                         current_pass_avg_acc,
                                         completed_data_passes + 1)

            epoch_running_loss = 0.0
            epoch_running_corrects = 0
            epoch_num_samples = 0
            completed_data_passes += 1
            train_iterator = iter(train_loader_val)
            batch = next(train_iterator)

        loss, accuracy = model_fn_speaker(batch, model_to_train, criterion_val,
                                          device_val)

        optimizer_val.zero_grad()
        loss.backward()
        optimizer_val.step()
        scheduler_val.step()

        current_lr = optimizer_val.param_groups[0]['lr']

        current_batch_size = batch[0].size(0)
        epoch_running_loss += loss.item() * current_batch_size
        epoch_running_corrects += accuracy.item() * current_batch_size
        epoch_num_samples += current_batch_size

        current_epoch_avg_train_acc = 0
        if epoch_num_samples > 0:
            current_epoch_avg_train_acc = epoch_running_corrects / epoch_num_samples

        if step % 100 == 0:
            tb_writer.add_scalar('Loss/train_batch', loss.item(), step)
            tb_writer.add_scalar('Accuracy/train_batch', accuracy.item(), step)
            tb_writer.add_scalar('LearningRate/step', current_lr, step)

        pbar_total_steps.set_postfix(
            batch_loss=f"{loss.item():.4f}",
            batch_acc=f"{accuracy.item():.2%}",
            run_avg_acc=f"{current_epoch_avg_train_acc:.2%}",
            lr=f"{current_lr:.2e}")

        if (step + 1) % valid_steps == 0:
            avg_valid_loss, avg_valid_acc = validate_speaker_model(
                valid_loader_val,
                model_to_train,
                criterion_val,
                device_val,
                step + 1,
                tb_writer,
                pbar_desc_prefix=f"Step {step+1}")

            pbar_total_steps.write(
                f"Step {step + 1}/{total_steps} - "
                f"Valid Loss: {avg_valid_loss:.4f}, Valid Acc: {avg_valid_acc:.2%}"
            )

            if avg_valid_acc > best_valid_acc:
                best_valid_acc = avg_valid_acc
                best_model_state_dict = model_to_train.state_dict()
                torch.save(best_model_state_dict, model_save_path)
                pbar_total_steps.write(
                    f"Best model updated at step {step + 1}. Accuracy: {best_valid_acc:.2%}. Saved to {model_save_path}"
                )
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= early_stop_patience:
                pbar_total_steps.write(
                    f"Early stopping triggered at step {step + 1} after {epochs_no_improve} validation checks without improvement."
                )
                break

    tb_writer.close()
    print(
        f"\nTraining finished. Best validation accuracy: {best_valid_acc:.4%}")
    if best_valid_acc > 0:
        print(f"Best model saved to {model_save_path}")
    else:
        print(
            "No model was saved as validation accuracy did not improve or training was too short."
        )

    return best_valid_acc


def train():
    # prepare dataloaders
    train_loader, valid_loader, speaker_num_from_data = get_dataloader(
        data_dir_val=data_dir,
        batch_size_val=batch_size,
        n_workers_val=num_workers)

    print(
        f"Train Dataloader: {len(train_loader.dataset)} samples, {len(train_loader)} batches"
    )
    print(
        f"Valid Dataloader: {len(valid_loader.dataset)} samples, {len(valid_loader)} batches"
    )

    # initialize model, criterion, optimizer, and scheduler
    print("Initializing model, criterion, optimizer, and scheduler...")
    speaker_model = Classifier(d_model=d_model,
                               dropout=dropout_rate,
                               num_speakers=num_speakers).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(speaker_model.parameters(), lr=learning_rate)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps,
                                                total_steps)

    # Run training
    best_accuracy_from_training = train_speaker_model(
        model_to_train=speaker_model,
        train_loader_val=train_loader,
        valid_loader_val=valid_loader,
        criterion_val=criterion,
        optimizer_val=optimizer,
        scheduler_val=scheduler,
        device_val=device)
    print(
        f"Training complete. Best validation accuracy: {best_accuracy_from_training:.2%}"
    )


# Training

In [59]:
train()


Train Dataloader: 50999 samples, 1593 batches
Valid Dataloader: 5667 samples, 178 batches
Initializing model, criterion, optimizer, and scheduler...


Total Training Steps:   0%|          | 0/70000 [00:00<?, ?it/s]

End of Training Data Pass 1: Avg Loss: 5.4047, Avg Acc: 4.62%


Step 2000 Validation:   0%|          | 0/178 [00:00<?, ?it/s]

Step 2000/70000 - Valid Loss: 4.0100, Valid Acc: 16.96%
Best model updated at step 2000. Accuracy: 16.96%. Saved to ./speaker_model.ckpt
End of Training Data Pass 2: Avg Loss: 3.8540, Avg Acc: 19.31%


...

Step 66000/70000 - Valid Loss: 1.3985, Valid Acc: 68.10%
End of Training Data Pass 42: Avg Loss: 1.0079, Avg Acc: 75.14%


Step 68000 Validation:   0%|          | 0/178 [00:00<?, ?it/s]

Step 68000/70000 - Valid Loss: 1.4175, Valid Acc: 67.85%
End of Training Data Pass 43: Avg Loss: 1.0048, Avg Acc: 75.44%


Step 70000 Validation:   0%|          | 0/178 [00:00<?, ?it/s]

Step 70000/70000 - Valid Loss: 1.4315, Valid Acc: 67.85%

Training finished. Best validation accuracy: 68.2548%
Best model saved to ./speaker_model.ckpt
Training complete. Best validation accuracy: 68.25%


## Tensorboard

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/  --port 6006


Reusing TensorBoard on port 6006 (pid 86512), started 2 days, 8:10:38 ago. (Use '!kill 86512' to kill it.)

# Test & Predict

In [17]:
# inference Dataset
class InferenceDataset(Dataset):

    def __init__(self, data_dir_val):
        testdata_path = Path(data_dir_val) / "testdata.json"
        metadata = json.load(testdata_path.open())
        self.data_dir = data_dir_val
        self.data = metadata["utterances"]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        utterance = self.data[index]
        feat_path = utterance["feature_path"]
        mel = torch.load(os.path.join(self.data_dir, feat_path))
        mel = torch.FloatTensor(mel)
        return feat_path, mel


# inference collate function
def inference_collate_batch(batch):
    feat_paths, mels = zip(*batch)
    mels_padded = pad_sequence(mels, batch_first=True, padding_value=-20)
    return feat_paths, mels_padded


# create submission file
def create_submission(device_val):
    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())

    dataset = InferenceDataset(data_dir)
    dataloader = DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        drop_last=False,
        num_workers=num_workers,
        collate_fn=inference_collate_batch,
    )
    print(f"[Info]: Finish loading data!", flush=True)

    speaker_num = len(mapping["id2speaker"])
    model = Classifier(num_speakers=speaker_num).to(device)
    model.load_state_dict(torch.load(model_save_path))
    model.eval()
    print(f"[Info]: Finish creating model!", flush=True)

    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            outs = model(mels)
            preds = outs.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])

    with open(submission_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)


# Run inference
create_submission(device)


[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/8000 [00:00<?, ?it/s]

e:\Anaconda\envs\pytorch\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
